### Извлекување на карактеристики од податоците на ниво на објект, месец (CustomerID, month). За секој објект и за секој месец се гради по еден вектор од карактеристики со следниве атрибути:
   - Број на планирани посети
   - Број на извршени планирани посети
   - Број на неизвршени планирани посети
   - Број на адхок посети
   - Денот (во месецот) на кој е направен најголем отстап во вредноста на SAPE во однос на претходната посета
   - Број на валидации (посети од контролор)
   - Средна вредност за SAPE коефициенти
   - Стандардна девијација за SAPE коефициенти
   - Средна вредност за APE коефициенти
   - Стандардна девијација за APE коефициенти
   - Квадратна разлика помеѓу средните вредности за SAPE и APE коефициентите
   - Средна вредност за FPE коефициенти
   - Стандардна девијација за FPE коефициенти
   - Број на 'ok' валидации
   - Број на 'not ok' валидации
   - Процент на 'ok' валидации
   - Процент на 'not ok' валидации


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('dataset/Spenser_1_7_remove_v22_treshold.tsv',
                 delimiter='\t')

In [3]:
def get_previous_not_nan_record(dates_sapes, i):
    if i-1 == 0:
        return dates_sapes[i-1][1]
    
    if not np.isnan(dates_sapes[i-1][1]):
        return dates_sapes[i-1][1]

    return get_previous_not_nan_record(dates_sapes, i-1)


def calculate_max_distance_day(df_month):
    max_index = 0
    max_distance = -float('inf')
    dates_sapes = []
    for date in list(df_month.Calendarday):
        dates_sapes.append((date, df_month[df_month.Calendarday == date].SAPECoeficient.values[0]))
    #dates_sapes = list(filter(lambda n: not np.isnan(n[1]), dates_sapes))
    
    first_not_non = np.nan
    date_first_not_nan = None
    for val in dates_sapes:
        if not np.isnan(val[1]):
            first_not_non = val[1]
            date_first_not_nan = val[0]
            break
    if np.isnan(first_not_non):
        return np.nan
    
    if all(map(lambda tup: tup[1] == first_not_non or np.isnan(tup[1]), dates_sapes)):
        return int(date_first_not_nan.split('-')[2])
    
    for i in range(1, len(dates_sapes) - 1):
        val_1 = dates_sapes[i][1]
        val_2 = get_previous_not_nan_record(dates_sapes, i)
        
        if np.isnan(val_1) or np.isnan(val_2):
            continue
        distance = (abs(val_1 - val_2))
        if distance > max_distance:
            max_index = i
            max_distance = distance
            
    max_distance_date = dates_sapes[max_index][0]
    return int(max_distance_date.split('-')[2])

In [4]:
expert_vectors = dict()
unique_customers = set(df.CustomerID)

for customer in unique_customers:
    df_customer = df[df.CustomerID == customer]
    months = set(df_customer.month)
    for month in months:
        df_customer_month = df_customer[df_customer.month == month]

        if len(df_customer_month) == 0:
            continue

        no_planned_visits = len(
            df_customer_month[df_customer_month.PerformanceEvaluationPlanned == '1'])  # број на планирани посети
        no_executed_visits = len(
            df_customer_month[df_customer_month.PerformanceEvaluationExecuted == '1'])  # број на извршени планирани посети
        no_not_executed_visits = no_planned_visits - no_executed_visits  # број на неизвршени планирани посети
        no_ad_hoc = len(df_customer_month[df_customer_month.PerformanceEvaluationAdHoc == '1'])  # број на адхок посети

        day_of_maximum_difference = calculate_max_distance_day(
            df_customer_month)  # на кој ден (во месецот) е настаната најголема разлика
        # во однос на претходната евалуација

        mean_sape = np.mean(df_customer_month.SAPECoeficient)  # средна вредност за SAPE коефициенти
        std_sape = np.std(df_customer_month.SAPECoeficient)  # стандардна девијација за SAPE коефициенти

        mean_fpe = np.mean(df_customer_month.FPECoeficient)  # средна вредност за FPE коефициенти
        std_fpe = np.std(df_customer_month.FPECoeficient)  # стандардна девијација за FPE коефициенти

        mean_ape = np.mean(df_customer_month.APECoeficient)  # средна вредност за APE коефициенти
        std_ape = np.std(df_customer_month.APECoeficient)  # стандардна девијација за APE коефициенти

        squared_err_sape_ape = (mean_sape - mean_ape) ** 2  # квадратна разлика помеѓу средните вредности за SAPE и APE

        no_ok_validations = len(df_customer_month[df_customer_month.OK == 'ok'])  # број на 'ok'валидации (OK == 'ok')
        no_not_ok_validations = len(
            df_customer_month[df_customer_month.NOTOK == 'not ok'])  # број на 'not ok' валидации (NOTOK == 'not ok')

        no_validations = len(
            df_customer_month[~np.isnan(df_customer_month.APECoeficient)])  # вкупен број на валидации (посети од контролор)

        percent_not_ok = float(
            no_not_ok_validations) / no_validations if no_validations != 0 else 0  # процент на 'ok' валидации
        percent_ok = float(
            no_ok_validations) / no_validations if no_validations != 0 else 0  # процент на 'not ok' валидации

        expert_vectors[(customer, month)] = [customer, month, no_planned_visits, no_executed_visits, no_not_executed_visits,
                                    no_ad_hoc, day_of_maximum_difference, no_validations, mean_sape,
                                    std_sape, mean_ape, std_ape, squared_err_sape_ape, mean_fpe, std_fpe,
                                    no_ok_validations, no_not_ok_validations, percent_ok, percent_not_ok]

In [5]:
cols = ['CustomerID', 'Month', 'No_planned_visits', 'No_executed_visits', 'No_not_executed_visits', 'No_adhoc_visits', 
        'Day_maximum_difference', 'No_validations' ,'Mean_SAPE', 'Std_dev_SAPE', 'Mean_APE', 'Std_dev_APE', 
        'Squared_error_SAPE_APE', 'Mean_FPE', 'Std_dev_FPE', 'No_ok_validations', 'No_not_ok_validations', 
        'Percent_ok', 'Percent_not_ok']

df_expert_vectors = pd.DataFrame(data=list(expert_vectors.values()), columns=cols)
df_expert_vectors = df_expert_vectors.sort_values(by=['CustomerID', 'Month'])
df_expert_vectors.to_csv('dataset/Expert_vectors_by_customerID_and_month.tsv', sep='\t', index=None)

In [6]:
df_expert_vectors

,CustomerID,Month,No_planned_visits,No_executed_visits,No_not_executed_visits,No_adhoc_visits,Day_maximum_difference,No_validations,Mean_SAPE,Std_dev_SAPE,Mean_APE,Std_dev_APE,Squared_error_SAPE_APE,Mean_FPE,Std_dev_FPE,No_ok_validations,No_not_ok_validations,Percent_ok,Percent_not_ok
17622,100000136,5,7,7,0,0,4.0,1,90.987000,1.421085e-14,79.498,0.0,131.997121,90.987,1.421085e-14,1,0,1.0,0.0
17623,100000136,6,8,8,0,0,15.0,0,92.632500,1.744089e+00,NaN,NaN,NaN,92.633,1.421085e-14,0,0,0.0,0.0
17624,100000136,7,4,4,0,0,24.0,0,95.375000,6.250000e-01,NaN,NaN,NaN,95.375,0.000000e+00,0,0,0.0,0.0
17625,100000136,8,8,8,0,0,31.0,0,96.000000,0.000000e+00,NaN,NaN,NaN,96.000,0.000000e+00,0,0,0.0,0.0
17626,100000136,9,9,9,0,0,31.0,0,96.000000,0.000000e+00,NaN,NaN,NaN,96.000,0.000000e+00,0,0,0.0,0.0
17627,100000136,10,6,6,0,0,2.0,1,96.000000,0.000000e+00,90.550,0.0,29.702500,96.000,0.000000e+00,1,0,1.0,0.0
17628,100000136,11,5,5,0,0,13.0,0,94.875000,2.500000e-01,NaN,NaN,NaN,94.875,0.000000e+00,0,0,0.0,0.0
17629,100000136,12,9,9,0,1,27.0,1,94.750000,0.000000e+00,70.499,0.0,588.111001,85.275,1.421085e-14,0,0,0.0,0.0
17630,100000141,5,7,7,0,0,15.0,0,43.518143,1.404287e+01,NaN,NaN,NaN,43.518,7.105427e-15,0,0,0.0,0.0
17631,100000141,6,8,8,0,0,22.0,0,52.830875,3.509371e+00,NaN,NaN,NaN,52.831,7.105427e-15,0,0,0.0,0.0
